In [49]:
import pandas as pd
import numpy as np
from pathlib import Path

In [50]:
raw_data_path: Path = Path("__file__").resolve().parents[1] / "data" / "raw" / "Assignment1.xlsx"
clean_data_dir: Path = Path("__file__").resolve().parents[1]/ "data" / "cleaned" 

## Accounts Table

In [51]:
raw_account_df: pd.DataFrame = pd.read_excel(raw_data_path, sheet_name="Accounts")

Problem in `raw_account_df` table:
- Name column includes `test` and `Test` characters which are unnecessary
- Makeing First and Last Name Capetalized

In [52]:
raw_account_df = raw_account_df.replace(to_replace="([Tt][Ee][Ss][Tt])+t*(ing)*[0-9]*", value="", regex=True)

In [53]:
def clean_names(x:str) -> str|float:
    """Take each name and clean them.
    Param:
        x: str = Name of each account holder
    Return:
        str or NaN(float)
    """
    name_list: list[str] = x.strip().split(" ")
    clean_names: str =  " ".join([name.capitalize() for name in name_list])
    if not clean_names: #if clean_names is empty
        return np.nan
    return clean_names


    
raw_account_df["Name"] = raw_account_df["Name"].map(clean_names)
raw_account_df["State"] = (
    raw_account_df["State"]
    .map(lambda x: "Maharashtra" if x == "MH" else x))

In [54]:
object_columns: list[str] = ["AccountId","Name","City","State"]

raw_account_df[object_columns] = raw_account_df[object_columns].fillna("Not Available")
raw_account_df["Pincode"] = raw_account_df["Pincode"].fillna(0)

## Policies Table

In [55]:
raw_policies_df: pd.DataFrame = pd.read_excel(raw_data_path, sheet_name="Policies")

There is one major problem with `raw_policies_df`.
1. Policy Name contains a lot of `Test` string with it.
2. It contains name of peoples in them which is out of scope for regex and requires NLP.

In [56]:
raw_policies_df = raw_policies_df.replace(to_replace="([Tt][Ee][Ss][Tt])+t*(ing)*[0-9]*",
                        value="",
                        regex=True)

raw_policies_df["Policy Name"] = raw_policies_df["Policy Name"].map(lambda x: " ".join(x.strip(" ").split(" ")))

## Claims Table

In [57]:
raw_claims_df: pd.DataFrame = pd.read_excel(raw_data_path, sheet_name="Claims")

There are 43 rows with `HAN` column as `NaN`. This means one of 2 things.
1. It is data collection error and the policy have not been registered.
2. They do not have any policy. 

We assume point number 2 and replace every cell with `NaN` values as Not Taken.

As per `Nan` in `BillAmount` Column we will replace it with value `0`.

In [58]:
raw_claims_df["HAN"] = raw_claims_df["HAN"].fillna("Not Taken")
raw_claims_df["BillAmount"] = raw_claims_df["BillAmount"].fillna(0)

## Saving CSV's
Saving to a single xlsx file with 3 sheets

In [59]:
from pandas import ExcelWriter
with ExcelWriter(clean_data_dir / "cleaned.xlsx") as writer:
    raw_account_df.to_excel(writer, sheet_name="Accounts", index=False)
    raw_policies_df.to_excel(writer, sheet_name="Policies", index=False)
    raw_claims_df.to_excel(writer, sheet_name="Claims", index=False)

In [60]:
# raw_account_df.to_csv(clean_data_dir / "account_cleaned.csv", index=False)
# raw_policies_df.to_csv(clean_data_dir / "policies_cleaned.csv", index=False)
# raw_claims_df.to_csv(clean_data_dir / "claims_cleaned.csv", index=False)